In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import heart_disease.preprocessing as pre_proc
import seaborn as sns
import pandas as pd
from copy import deepcopy as cp
%load_ext autoreload

%autoreload 2



In [2]:
# Import Raw Data

data_file_name_str = 'processed.cleveland.data.txt'

data_df = pre_proc.load_raw_data(data_file_name_str)

data_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,ispos_truth
0,63.0,True,1.0,145.0,233.0,True,2.0,150.0,False,2.3,3.0,0.0,6.0,False
1,67.0,True,4.0,160.0,286.0,False,2.0,108.0,True,1.5,2.0,3.0,3.0,True
2,67.0,True,4.0,120.0,229.0,False,2.0,129.0,True,2.6,2.0,2.0,7.0,True
3,37.0,True,3.0,130.0,250.0,False,0.0,187.0,False,3.5,3.0,0.0,3.0,False
4,41.0,False,2.0,130.0,204.0,False,2.0,172.0,False,1.4,1.0,0.0,3.0,False
